In [40]:
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse

import util

In [41]:
def extract_feats(ffs, direc="train", global_feat_dict=None):
    """
    arguments:
      ffs are a list of feature-functions.
      direc is a directory containing xml files (expected to be train or test).
      global_feat_dict is a dictionary mapping feature_names to column-numbers; it
      should only be provided when extracting features from test data, so that 
      the columns of the test matrix align correctly.

    returns: 
      a sparse design matrix, a dict mapping features to column-numbers,
      a vector of target classes, and a list of system-call-history ids in order 
      of their rows in the design matrix.
      
      Note: the vector of target classes returned will contain the true indices of the
      target classes on the training data, but will contain only -1's on the test
      data
    """
    fds = [] # list of feature dicts
    classes = []
    ids = [] 
    for datafile in os.listdir(direc):
        # extract id and true class (if available) from filename
        id_str,clazz = datafile.split('.')[:2]
        ids.append(id_str)
        # add target class if this is training data
        try:
            classes.append(util.malware_classes.index(clazz))
        except ValueError:
            # we should only fail to find the label in our list of malware classes
            # if this is test data, which always has an "X" label
            assert clazz == "X"
            classes.append(-1)
        rowfd = {}
        # parse file as an xml document
        tree = ET.parse(os.path.join(direc,datafile))
        # accumulate features
        [rowfd.update(ff(tree)) for ff in ffs]
        fds.append(rowfd)
    
    X,feat_dict = make_design_mat(fds,global_feat_dict)
    return X, feat_dict, np.array(classes), ids

In [42]:
def make_design_mat(fds, global_feat_dict=None):
    """
    arguments:
      fds is a list of feature dicts (one for each row).
      global_feat_dict is a dictionary mapping feature_names to column-numbers; it
      should only be provided when extracting features from test data, so that 
      the columns of the test matrix align correctly.
       
    returns: 
        a sparse NxD design matrix, where N == len(fds) and D is the number of
        the union of features defined in any of the fds 
    """
    if global_feat_dict is None:
        all_feats = set()
        [all_feats.update(fd.keys()) for fd in fds]
        feat_dict = dict([(feat, i) for i, feat in enumerate(sorted(all_feats))])
    else:
        feat_dict = global_feat_dict
    
    cols = []
    rows = []
    data = []        
    for i in xrange(len(fds)):
        temp_cols = []
        temp_data = []
        for feat,val in fds[i].iteritems():
            try:
                # update temp_cols iff update temp_data
                temp_cols.append(feat_dict[feat])
                temp_data.append(val)
            except KeyError as ex:
                if global_feat_dict is not None:
                    pass  # new feature in test data; nbd
                else:
                    raise ex

        # all fd's features in the same row
        k = len(temp_cols)
        cols.extend(temp_cols)
        data.extend(temp_data)
        rows.extend([i]*k)

    assert len(cols) == len(rows) and len(rows) == len(data)
   

    X = sparse.csr_matrix((np.array(data),
                   (np.array(rows), np.array(cols))),
                   shape=(len(fds), len(feat_dict)))
    return X, feat_dict

In [43]:
## Here are two example feature-functions. They each take an xml.etree.ElementTree object, 
# (i.e., the result of parsing an xml file) and returns a dictionary mapping 
# feature-names to numeric values.
## TODO: modify these functions, and/or add new ones.
def first_last_system_call_feats(tree):
    """
    arguments:
      tree is an xml.etree.ElementTree object
    returns:
      a dictionary mapping 'first_call-x' to 1 if x was the first system call
      made, and 'last_call-y' to 1 if y was the last system call made. 
      (in other words, it returns a dictionary indicating what the first and 
      last system calls made by an executable were.)
    """
    c = Counter()
    in_all_section = False
    first = True # is this the first system call
    last_call = None # keep track of last call we've seen
    for el in tree.iter():
        # ignore everything outside the "all_section" element
        if el.tag == "all_section" and not in_all_section:
            in_all_section = True
        elif el.tag == "all_section" and in_all_section:
            in_all_section = False
        elif in_all_section:
            if first:
                c["first_call-"+el.tag] = 1
                first = False
            last_call = el.tag  # update last call seen
            
    # finally, mark last call seen
    c["last_call-"+last_call] = 1
    return c

In [44]:
def system_call_count_feats(tree):
    """
    arguments:
      tree is an xml.etree.ElementTree object
    returns:
      a dictionary mapping 'num_system_calls' to the number of system_calls
      made by an executable (summed over all processes)
    """
    c = Counter()
    in_all_section = False
    for el in tree.iter():
        # ignore everything outside the "all_section" element
        if el.tag == "all_section" and not in_all_section:
            in_all_section = True
        elif el.tag == "all_section" and in_all_section:
            in_all_section = False
        elif in_all_section:
            c['num_system_calls'] += 1
    return c

In [45]:
#############################
#
#  Below is main function
#
#############################

In [46]:
train_dir = "train"
test_dir = "test"
outputfile = "experiment_predictions.csv"  # feel free to change this or take it as an argument

In [47]:
# TODO put the names of the feature functions you've defined above in this list
ffs = [first_last_system_call_feats, system_call_count_feats]

In [48]:
# extract features
print "extracting training features..."
X_train,global_feat_dict,t_train,train_ids = extract_feats(ffs, train_dir)

extracting training features...


In [49]:
y_train = np.zeros((len(t_train),len(util.malware_classes)))
y_train[np.arange(len(t_train)), t_train] = 1
y_train.shape

(3086, 15)

In [50]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(class_weight = "balanced", n_jobs = -1)
RF.fit(X_train, y_train)
print RF.n_features_
print RF.feature_importances_

31
[0.00000000e+00 9.80387572e-04 3.23302367e-03 2.12694559e-03
 1.56771318e-02 2.89670942e-03 1.98475648e-04 2.38986389e-02
 3.62888883e-04 3.31895642e-03 1.64414698e-04 1.16206134e-03
 4.75763677e-05 2.29913862e-02 1.50058415e-03 1.16636365e-02
 7.75394113e-04 1.98879905e-03 5.86438986e-05 2.01217181e-02
 1.21691346e-02 7.88750495e-03 1.65000090e-05 1.42656596e-04
 3.60735327e-03 8.12440033e-05 1.06915421e-02 1.44513423e-02
 1.55672263e-03 4.40079131e-06 8.36224226e-01]


In [51]:
import operator
sorted_global_feat_dict = sorted(global_feat_dict.items(), key=operator.itemgetter(1))
print sorted_global_feat_dict

[('first_call-load_image', 0), ('last_call-check_for_debugger', 1), ('last_call-create_mutex', 2), ('last_call-create_open_file', 3), ('last_call-create_thread', 4), ('last_call-create_window', 5), ('last_call-dump_line', 6), ('last_call-enum_window', 7), ('last_call-find_file', 8), ('last_call-get_file_attributes', 9), ('last_call-get_host_by_name', 10), ('last_call-get_system_directory', 11), ('last_call-get_system_time', 12), ('last_call-kill_process', 13), ('last_call-listen_socket', 14), ('last_call-load_dll', 15), ('last_call-load_driver', 16), ('last_call-open_file', 17), ('last_call-open_key', 18), ('last_call-open_process', 19), ('last_call-process', 20), ('last_call-query_value', 21), ('last_call-read_value', 22), ('last_call-set_value', 23), ('last_call-set_windows_hook', 24), ('last_call-show_window', 25), ('last_call-sleep', 26), ('last_call-thread', 27), ('last_call-vm_protect', 28), ('last_call-write_value', 29), ('num_system_calls', 30)]


In [52]:
RF.score(X_train, y_train)

0.8885288399222294

In [53]:
# get rid of training data and load test data
# del X_train
# del t_train
# del train_ids

In [54]:
print "extracting test features..."
X_test,_,t_ignore,test_ids = extract_feats(ffs, test_dir, global_feat_dict=global_feat_dict)

extracting test features...


In [55]:
preds = RF.predict(X_test)

In [56]:
preds[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [57]:
# TODO make predictions on text data and write them out
print "making predictions..."
results = np.argmax(preds, axis=1)

print "writing predictions..."
util.write_predictions(results, test_ids, outputfile)

making predictions...
writing predictions...


In [58]:
import csv 
import sys 

def reorder_submission(file_to_reorder, newfile_name = "experiment_results.csv"):
    # READ IN KEYS IN CORRECT ORDER AS LIST
    with open('keys.csv','r') as f:
        keyreader = csv.reader(f)
        keys = [key[0] for key in keyreader]

    # READ IN ALL PREDICTIONS, REGARDLESS OF ORDER
    with open(file_to_reorder) as f:
        oldfile_reader = csv.reader(f)
        D = {}
        for i,row in enumerate(oldfile_reader):
            if i == 0:
                continue
            _id, pred = row 
            D[_id] = pred

    # WRITE PREDICTIONS IN NEW ORDER
    with open(newfile_name,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('Id','Prediction'))
        for key in keys:
            writer.writerow((key,D[key]))

    print("".join(["Reordered ", file_to_reorder," and wrote to ", newfile_name]))

In [59]:
reorder_submission(outputfile, "experiment_randomforest_results_2.csv")

Reordered experiment_predictions.csv and wrote to experiment_randomforest_results_2.csv
